#### This file is to quickly review the trained summarize model output.
##### Debug and Development purpose only

In [19]:
import torch
from datasets import load_dataset, load_metric
from transformers import AutoTokenizer
from transformers import AutoModelForSeq2SeqLM
import json
import random
import numpy as np
import datasets as hf_datasets
from torch.utils.data import DataLoader
from transformers import pipeline
from tqdm import tqdm
import evaluate
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
import nltk

from nltk.tokenize import sent_tokenize
from transformers import DataCollatorForSeq2Seq

Choose the model to load

In [20]:
checkpoint = './t5-base-finetuned-20230907/checkpoint-46500'
summarizer = pipeline("summarization", tokenizer=checkpoint, framework='pt', model=checkpoint)

Choose dataset to load

In [3]:
from T5_only_summarize_demo_train import read_in_dataset
mydataset = read_in_dataset()
mydataset['summary'] = [k['0'] for k in mydataset['summary']]
mydataset = hf_datasets.Dataset.from_dict(mydataset)
train_test = mydataset.train_test_split(test_size=0.2, seed=66)
train_val = train_test['train'].train_test_split(test_size=0.2, seed=88)
#Need to do test on testing data
testing_data = train_test['test']
del train_test, train_val

d:\AILab_env\lib\site-packages\transformers\models\t5\tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\z003t1zx\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Evaluate on one single random data from testing

In [9]:
sample = testing_data.select(range(1117,1118))
suid = sample['suid'][0]
paragraph = 'summarize: ' + sample['text'][0]
gt = sample['summary'][0]
predicted = summarizer(paragraph)

print('Predicted Finetune: ')
print("##############################")
print(predicted[0]['summary_text'])
print('Ground Truth: ')
print("##############################")
print(gt)
print("##############################")
print('original_text')
print(sample['text'][0])

Predicted Finetune: 
##############################
Your CT scan showed some thickening in the right lung, fluid, and a mass or hematoma in the right spinal area. The ascending aorta is mildly enlarged, and there are calcified lymph nodes in the hila. Further evaluation with a bone scan, PET/CT, and MRI is recommended.
Ground Truth: 
##############################
The CT scan shows some pleural thickening and reticular lines in the right chest base, with mild fibrosis. There is also pleural thickening, fluid, and possible atelectasis. The left base has similar findings but to a lesser extent. No worrisome nodules or masses were observed. There is a lytic destructive process in the 10th vertebral body, and further evaluation is recommended. 
##############################
original_text
\n**REMOVED,_P.C.\n**HEADERFIELD\n**HEADERFIELD\n**HEADERFIELD\nExamination Report\nTo:**HEADERFIELD\n**HEADERFIELD\n**HEADERFIELD\n**HEADERFIELD\nFax#:    \nExam#: 404679 - 71250 71250 CT CHEST W/O\nCT S

Baseline without training

In [10]:
checkpoint_baseline = 't5-base'
summarizer_base = pipeline("summarization", tokenizer=checkpoint_baseline, framework='pt', model=checkpoint_baseline)
predicted_base = summarizer_base(paragraph)
print('Predicted Baseline: ')
print("***************************")
print(predicted_base[0]['summary_text'])
print("***************************")
print(gt)

Token indices sequence length is longer than the specified maximum sequence length for this model (803 > 512). Running this sequence through the model will result in indexing errors


Predicted Baseline: 
***************************
chest x-ray shows increasednmarkings . no worrisome nodule or Parenchymal mass seen . pleural thickening noted on the right with subpleural lines .
***************************
The CT scan shows some pleural thickening and reticular lines in the right chest base, with mild fibrosis. There is also pleural thickening, fluid, and possible atelectasis. The left base has similar findings but to a lesser extent. No worrisome nodules or masses were observed. There is a lytic destructive process in the 10th vertebral body, and further evaluation is recommended. 


## T5 not using pipeline (multi-task)

In [7]:
from Data import prepare
import json
jsonfile = "F:/SmartReport/GT.json"
trimmed_dataset = prepare.read_in_dataset(jsonfile=jsonfile)
with open("F:/SmartReport/split_2023-09-19_11-44-08.json") as fid:
    train_val_test_split = json.load(fid)
test_suids = set(train_val_test_split['test'])

#### Build test data dictionary

In [10]:
from collections import defaultdict
testing_dict = defaultdict(defaultdict)
for idx in range(len(trimmed_dataset['suid'])):
    suid = trimmed_dataset['suid'][idx]
    if suid in test_suids:
        testing_dict[suid]['context'] = trimmed_dataset['text'][idx]
        testing_dict[suid]['summary'] = trimmed_dataset['summary'][idx]
        testing_dict[suid]['malignancy'] = trimmed_dataset['malignancy'][idx]
        

### end2end prediction multi task 

In [40]:
from transformers import T5Tokenizer, T5ForConditionalGeneration  
  
# Initialize the tokenizer and model  
tokenizer = T5Tokenizer.from_pretrained('t5-base', model_max_length=1024, padding='max_length')  
model = T5ForConditionalGeneration.from_pretrained('F:/SmartReport/Training_snapshots/t5-base-finetuned/checkpoint-3000')  
  
# Define the text  
selected_suid = list(testing_dict.keys())[0]
text = 'summarize: ' + testing_dict[selected_suid]['context']
#text = 'question: is there cancer or malignant nodule? ' +'context: '+ testing_dict[selected_suid]['context']
  
# Tokenize the text  
input_ids = tokenizer.encode(text, return_tensors='pt')  
  
# Generate output  
output = model.generate(input_ids, max_length=200)  
  
# Decode the output  
decoded_output = tokenizer.decode(output[0])  
  
print(decoded_output)  
